In [1]:
# Incase Of Update
# response = requests.get('https://db.ygoprodeck.com/api/v7/cardinfo.php')
# json_response = response.json()
# dataset = pd.DataFrame(json_response['data'])

# dataset.to_csv('Dataset/Yugioh_Database.csv')

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings

import random as random
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


warnings.simplefilter(action='ignore', category=FutureWarning)
plt.style.use('Solarize_Light2')
pd.set_option('display.max_columns', 20)

#import requests
#import itertools

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0] , True)


In [3]:
# ========================================================================
'''Load Dataset'''
dataset = pd.read_csv('Dataset/Yugioh_Database.csv')
dataset = dataset.drop(['Unnamed: 0' , 'frameType' , 'archetype' , 'ygoprodeck_url' , 'card_sets' , 'card_images' , 'card_prices' , 'banlist_info'],axis=1)
dataset = dataset[dataset['type'] != 'XYZ Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Synchro Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Fusion Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Pendulum Effect Monster']
dataset = dataset[dataset['type'] != 'Pendulum Normal Monster']

dataset = dataset[dataset['type'] != 'Skill Card']
dataset = dataset[dataset['type'] != 'Monster Token']
	 
# dataset = dataset[dataset['type'] != 'Spell Card']
# dataset = dataset[dataset['type'] != 'Trap Card']
dataset = dataset[dataset['type'] != 'Fusion Monster']
dataset = dataset[dataset['type'] != 'XYZ Monster']
dataset = dataset[dataset['type'] != 'Synchro Monster']
dataset = dataset[dataset['type'] != 'Link Monster']
# Staple Removal
dataset = dataset[[not i for i in dataset['name'].isin(['Ash Blossom & Joyous Spring' , 'Effect Veiler' , 'Ghost Ogre & Snow Rabbit' ,'Ghost Belle & Haunted Mansion',
                                                        'Infinite Impermanence' , 'Red Reboot' , 'Called by the Grave' , 'Forbidden Droplet' , 'Crossout Designator',
                                                        'Nibiru, the Primal Being', 'Harpie\"s Feather Duster' , 'Lightning Storm' , 'Pot of Prosperity' , 'Pot of Desires',
                                                        'Pot of Duality' , 'Pot of Extravagance' , 'Triple Tactics Talents' , 'Torrential Tribute' , 'Dark Ruler No More' , 
                                                        'Red Reboot', 'D.D. Crow' , 'PSY-Framegear Gamma' , 'Maxx \"C\"' , 'Dimension Shifter' , 'Droll & Lock Bird' , 
                                                        'Accesscode Talker', 'Apollousa, Bow of the Goddess', 'Borreload Dragon' , 'Borrelsword Dragon', 'Knightmare Unicorn',
                                                        'Predaplant Verte Anaconda' , 'Knightmare Phoenix' , 'Knightmare Cerberus' , 'Underworld Goddess of the Closed World',
                                                        'Borreload Savage Dragon' , 'Token Collector' , 'Evenly Matched' , 'Forbidden Chalice' , 'Cosmic Cyclone' , 'Contact \"C\"',
                                                        'Retaliating \"C\"' , 'Gadarla, the Mystery Dust Kaiju' , 'Solemn Judgment' , 'Dimensional Barrier' , 'Solemn Strike',
                                                         'Ice Dragon\'s Prison' , 'Gozen Match' ])]]

dataset.loc[dataset['type']=='Normal Monster', ['desc']] = 'NoInfo'
dataset = dataset.fillna('0')
dataset['level'] = dataset['level'].astype('int32')



# ========================================================================
'''Create Tokenized sequence database'''


df = dataset['desc']         #Tokenizer is only trained on desc and based on that . Otherwise if trained on names it would blow vocab up to absurd amounts
Sliced_df = dataset[['level' , 'race' , 'type' , 'attribute' , 'name' , 'desc']]

for i in range(1,11,2):
    Sliced_df.insert(loc=i, column='A'+str(i), value=-1)        # Adds seperator columns


Sliced_df = Sliced_df.reset_index(drop=True)            # Need to reset the indexes so they are consistent
df = df.reset_index(drop=True)                              

tokenizer = Tokenizer(filters='\r , \n , \" ') # Speech marks stop names from being recognised by tokenizer
tokenizer.fit_on_texts(df)
tokenizer.word_index['0'] = 0           #Signifies Empty values
tokenizer.word_index['-1'] = -1           #Signifies Seperators

sequences = []
padded_sequences = []
Tokenized_sequence_database = []
count = 0
for i in Sliced_df.astype('string').to_numpy():
    
    sequences.append(tokenizer.texts_to_sequences(i))
    

for i in range(0,11):
    padded_sequences.append( pad_sequences(np.array(sequences , dtype='object')[:,i], padding='post') ) 

Tokenized_sequence_database = np.concatenate(([padded_sequences[i] for i in range(11)]) , axis=1 )

    




In [4]:
Sliced_df

,level,A1,race,A3,type,A5,attribute,A7,name,A9,desc
0,0,-1,Continuous,-1,Spell Card,-1,0,-1,"""A"" Cell Breeding Device",-1,"During each of your Standby Phases, put 1 A-Co..."
1,0,-1,Continuous,-1,Spell Card,-1,0,-1,"""A"" Cell Incubator",-1,Each time an A-Counter(s) is removed from play...
2,0,-1,Quick-Play,-1,Spell Card,-1,0,-1,"""A"" Cell Recombination Device",-1,Target 1 face-up monster on the field; send 1 ...
3,0,-1,Quick-Play,-1,Spell Card,-1,0,-1,"""A"" Cell Scatter Burst",-1,"Select 1 face-up ""Alien"" monster you control. ..."
4,0,-1,Equip,-1,Spell Card,-1,0,-1,"""Infernoble Arms - Almace""",-1,While this card is equipped to a monster: You ...
...,...,...,...,...,...,...,...,...,...,...,...
10685,4,-1,Beast,-1,Effect Monster,-1,LIGHT,-1,ZW - Sleipnir Mail,-1,"You can target 1 ""Utopia"" monster you control;..."
10686,4,-1,Beast,-1,Effect Monster,-1,LIGHT,-1,ZW - Sylphid Wing,-1,"You can only control 1 ""ZW - Sylphid Wing"". Yo..."
10687,5,-1,Dragon,-1,Effect Monster,-1,WIND,-1,ZW - Tornado Bringer,-1,"You can target 1 ""Utopia"" monster you control;..."
10688,4,-1,Aqua,-1,Effect Monster,-1,EARTH,-1,ZW - Ultimate Shield,-1,When this card is Normal or Special Summoned: ...


In [5]:
def Deck_Loader(directory):
    '''Loads Decks from Deck_Lists.txt as arrays and stores those arrays in altered'''
    file = open(directory , 'r')
    read = file.readlines()
    Deck_Array = []
    flag = False

    temp=[]

    for count,line in enumerate(read):
        
        if '//' in read[count]:
            flag = not flag
        
        if flag:
            
            read[count] = read[count].replace('\n','')
            
            if ('=='  in read[count]) or ('//'  in read[count])  :
                pass
                
            else:
                for i in range(int(read[count][0])):
                    temp.append(read[count][1:].strip())          #skip appending also remove white space

        if (not flag) or (count == len(read) - 1):
            Deck_Array.append(temp)
            temp = []
            flag = not flag
            
            
    file.close()
    return Deck_Array 

def Dataset_Builder(Direc):
    Loaded_Decks = Deck_Loader(Direc)

    Output = []
    for Deck in Loaded_Decks:
        for _ in range(30):
            choices = Sliced_df[Sliced_df['name'].isin(Deck)].index.values          # Prevents staples from leaking into data as sliceddf is filtered from staples.

            indexes = random.sample(choices.tolist() , 6)
            Decider = [Tokenized_sequence_database[i] for i in indexes[0:5]]
            Subject = [Tokenized_sequence_database[indexes[5]] , np.zeros(181 , dtype='int32') , np.zeros(181 , dtype='int32') , np.zeros(181 , dtype='int32') , np.zeros(181 , dtype='int32')]
            Output.append([Decider , Subject , [1]])

    for _ in range(len(Output)):
        Decks = random.sample(Loaded_Decks , 2)
        choices1 = Sliced_df[Sliced_df['name'].isin(Decks[0])].index.values 
        choices2 = Sliced_df[Sliced_df['name'].isin(Decks[1])].index.values
        
        Decider = [Tokenized_sequence_database[i] for i in random.sample(choices1.tolist() , 5)]
        Subject = [Tokenized_sequence_database[random.choice(choices2)] , np.zeros(181 , dtype='int32') , np.zeros(181 , dtype='int32') , np.zeros(181 , dtype='int32') , np.zeros(181 , dtype='int32')]
        Output.append([Decider , Subject , [0]])
    
    random.shuffle(Output)
    random.shuffle(Output)
    random.shuffle(Output)
    return Output

Training_Dataset = Dataset_Builder('Dataset/Training_Deck_Lists.txt')
Validation_Dataset = Dataset_Builder('Dataset/Validation_Deck_Lists.txt')


#pd.DataFrame(Training_Dataset)
#x,y = Training_Dataset[0][0]
#tokenizer.sequences_to_texts(np.array(y))
#np.array(x).shape        

In [17]:
def positional_encoding(length, depth):
  depth = depth/2
  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate( [np.sin(angle_rads), np.cos(angle_rads)], axis=-1) 

  return pos_encoding


class Custom_Embedder(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()

    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(vocab_size), 
      tf.keras.layers.Dense(d_model )
      
    ])
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self,x):

    x = self.seq(x)
    x = self.layer_norm(x)
    
    return x

class Positional_Embedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()

    self.d_model = d_model
    
    self.embedding = Custom_Embedder(vocab_size, d_model) 
    self.pos_encoding = positional_encoding(length = 5, depth=d_model)
    
  def call(self, x):
    x = self.embedding(x)
    
    x*= np.sqrt(self.d_model) # Scale Values by their embedding dimensionality otherwise they could get overwhelmed by positional encoder
    x = x + self.pos_encoding 
    return x





class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class DeciderSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, ffn, dropout_rate):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(ffn, activation='relu'), 
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x
  
class EncoderLayer(tf.keras.layers.Layer):
  """Single Encoder Layer with DeciderMHA and Feed Forward layer"""
  def __init__(self, d_model, ffn , dropout_rate ):
    super().__init__()

    self.DSA = DeciderSelfAttention(num_heads=12, key_dim=d_model , dropout = dropout_rate)       # Scaling Number of Heads increases parameters as this is a different implementation of mha compared to attention is all you need paper.
    self.FF = FeedForward(d_model, ffn , dropout_rate)

  def call(self, x):
    
    
    x = self.DSA(x)

    x = self.FF(x)

    return x

class Encoder(tf.keras.layers.Layer):
  """Full Encoder with embedding layer with dropout and encoder layers"""
  def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.num_layers = num_layers

    self.Pos_Embedding = Positional_Embedding(vocab_size, d_model)
    self.EL = [EncoderLayer(d_model, ffn , dropout_rate) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self,x):
    
    x = self.Pos_Embedding(x)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.EL[i](x)

    return x



class SubjectSelfAttention(BaseAttention):
  
  def call(self, x):
    
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class SubjectCrossAttention(BaseAttention):

  def call(self, x , context):
    attn_output = self.mha(
        query=x,
        value=context,
        key=context)
    
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

class ComparatorLayer(tf.keras.layers.Layer):

  def __init__(self , d_model, ffn , dropout_rate):
    super().__init__()

    self.SSA = SubjectSelfAttention(num_heads=12, key_dim=d_model , dropout = dropout_rate)      
    self.SCA = SubjectCrossAttention(num_heads=12, key_dim=d_model , dropout = dropout_rate)
    self.FF = FeedForward(d_model, ffn , dropout_rate)

  def call(self, x , context):
    
    x = self.SSA(x)

    x = self.SCA(x , context)

    x = self.FF(x)

    return x
  
class Comparator(tf.keras.layers.Layer):
  
  def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.num_layers = num_layers

    self.Pos_Embedding = Positional_Embedding(vocab_size, d_model)
    self.CL = [ComparatorLayer(d_model, ffn , dropout_rate) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x , context):
    
    x = self.Pos_Embedding(x)
    
    x = self.dropout(x)
    
    for i in range(self.num_layers):
      x = self.CL[i](x , context)

    return x

class FinalFeedForward(tf.keras.layers.Layer):
  def __init__(self , dropout_rate):
    super().__init__()

    self.seq = tf.keras.Sequential([
      tf.keras.layers.Flatten(),          #Flattens sentances for each card comparision , into a single 1d array , so it can generate probabilities properly, instead of shoving 100 x905 matrix straight through and generating 100 probabilities for each card comparision feature embedding
      # tf.keras.layers.Dense(50, activation='relu'),
      # tf.keras.layers.Dense(25, activation='relu'),
      # tf.keras.layers.Dropout(dropout_rate),
      tf.keras.layers.Dense(1 , activation='sigmoid')
      
    ])
    
  def call(self, x):
    
    x = self.seq(x) 
    return x
  
class FullModel(tf.keras.Model):
   def __init__(self, d_model, vocab_size , ffn , dropout_rate , num_layers):
    super().__init__()

    self.enc = Encoder(d_model, vocab_size , ffn , dropout_rate , num_layers)
    self.com = Comparator(d_model, vocab_size , ffn , dropout_rate , num_layers)
    self.FFF = FinalFeedForward(dropout_rate)

   def call(self, inputs):
     context , x = inputs
     
     
     
     context = self.enc(context)
    
     x = self.com(x , context)

     x = self.FFF(x)

     return x
   
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [18]:
def Train_Gen():
    seti0 = [i[0] for i in Training_Dataset]
    seti1 = [i[1] for i in Training_Dataset]
    seti2 = [i[2] for i in Training_Dataset]
    for _ in range(10000):
        r_int = random.randint(0 , len(Training_Dataset) - 50)
        train_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+50])
        train_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+50])
        train_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+50])
    
        yield (train_seti0,train_seti1), train_seti2

def Val_Gen():
    seti0 = [i[0] for i in Validation_Dataset]
    seti1 = [i[1] for i in Validation_Dataset]
    seti2 = [i[2] for i in Validation_Dataset]
    for _ in range(10000):
        r_int = random.randint(0 , len(Validation_Dataset) - 50)
        val_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+50])
        val_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+50])
        val_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+50])

        yield (val_seti0,val_seti1), val_seti2

def Train_Split_Gen():
    seti0 = [i[0] for i in Training_Dataset]
    seti1 = [i[1] for i in Training_Dataset]
    seti2 = [i[2] for i in Training_Dataset]
    for _ in range(10000):
        r_int = random.randint(0 , int(len(Training_Dataset)*0.8) - 50)
        train_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+50])
        train_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+50])
        train_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+50])
    
        yield (train_seti0,train_seti1), train_seti2

def Val_Split_Gen():
    seti0 = [i[0] for i in Training_Dataset]
    seti1 = [i[1] for i in Training_Dataset]
    seti2 = [i[2] for i in Training_Dataset]
    for _ in range(10000):
        r_int = random.randint(int(len(Training_Dataset)*0.8) , len(Training_Dataset) - 50)
        train_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+50])
        train_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+50])
        train_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+50])
    
        yield (train_seti0,train_seti1), train_seti2

def Pred_Split_Gen():
    seti0 = [i[0] for i in Training_Dataset]
    seti1 = [i[1] for i in Training_Dataset]
    seti2 = [i[2] for i in Training_Dataset]
    for _ in range(1):
        r_int = random.randint(int(len(Training_Dataset)*0.8) , len(Training_Dataset) - 300)
        train_seti0 = tf.convert_to_tensor(seti0[r_int:r_int+300])
        train_seti1 = tf.convert_to_tensor(seti1[r_int:r_int+300])
        train_seti2 = tf.convert_to_tensor(seti2[r_int:r_int+300])
    
        yield (train_seti0,train_seti1), train_seti2

In [16]:
Model = FullModel(200 , 5000 , 1000 , 0.3 , 1)

learning_rate = CustomSchedule(d_model = 200)

optimizer = tf.keras.optimizers.Adam(learning_rate , beta_1=0.9, beta_2=0.98, epsilon=1e-9)

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir='Logs6' ,histogram_freq=1
)

Model.compile(
    loss= tf.keras.losses.BinaryCrossentropy(),
    optimizer=optimizer,
    metrics= 'accuracy' )

history = Model.fit(Train_Split_Gen() , epochs=50, 
                               validation_data = Val_Split_Gen()  , steps_per_epoch=100 , batch_size=50 , 
                               validation_steps=50 , validation_batch_size=50 , callbacks=[tensorboard_callback])

Epoch 1/50
100/100 [==============================] - 7s 43ms/step - loss: 0.7468 - accuracy: 0.5162 - val_loss: 0.7317 - val_accuracy: 0.4980
Epoch 2/50
100/100 [==============================] - 4s 37ms/step - loss: 0.6911 - accuracy: 0.5848 - val_loss: 0.8071 - val_accuracy: 0.4828
Epoch 3/50
100/100 [==============================] - 4s 36ms/step - loss: 0.5653 - accuracy: 0.7012 - val_loss: 1.0701 - val_accuracy: 0.4876
Epoch 4/50
100/100 [==============================] - 4s 36ms/step - loss: 0.5087 - accuracy: 0.7590 - val_loss: 1.2723 - val_accuracy: 0.4972
Epoch 5/50
100/100 [==============================] - 4s 36ms/step - loss: 0.3745 - accuracy: 0.8408 - val_loss: 1.1539 - val_accuracy: 0.5140
Epoch 6/50
100/100 [==============================] - 4s 36ms/step - loss: 0.4025 - accuracy: 0.8214 - val_loss: 1.1763 - val_accuracy: 0.4764
Epoch 7/50
100/100 [==============================] - 4s 36ms/step - loss: 0.3969 - accuracy: 0.8274 - val_loss: 1.3252 - val_accuracy: 0.4828

In [30]:
Model.summary()

Model: "full_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_3 (Encoder)         multiple                  3999800   
                                                                 
 comparator_3 (Comparator)   multiple                  5285800   
                                                                 
 final_feed_forward_3 (Final  multiple                 1001      
 FeedForward)                                                    
                                                                 
Total params: 9,286,601
Trainable params: 9,286,601
Non-trainable params: 0
_________________________________________________________________


In [92]:
### Plot prediction values against true Labels
Data , True_Labels = next(Pred_Split_Gen())
Predictions = Model.predict(Data)

vals = pd.DataFrame([{'z' : 0 ,'x' : i[0],'y' : j[0]} for i,j in zip(Predictions , True_Labels.numpy())])

#sns.scatterplot(data=vals , x='z' , y='x' , hue='y')

for threshold in range(0,20,1):
    count = 0
    for i in vals.values:
        if i[1] > threshold/20:
            pred = 1
        else:
            pred = 0
        
        if pred == i[2]:
            count+=1
    print('Accuracy is: ' + str(count/len(vals)) + 'threshold is: ' + str(threshold/20) )

10/10 [==============================] - 0s 12ms/step
Accuracy is: 0.47333333333333333threshold is: 0.0
Accuracy is: 0.4766666666666667threshold is: 0.05
Accuracy is: 0.4766666666666667threshold is: 0.1
Accuracy is: 0.48threshold is: 0.15
Accuracy is: 0.49666666666666665threshold is: 0.2
Accuracy is: 0.49threshold is: 0.25
Accuracy is: 0.5166666666666667threshold is: 0.3
Accuracy is: 0.52threshold is: 0.35
Accuracy is: 0.5433333333333333threshold is: 0.4
Accuracy is: 0.52threshold is: 0.45
Accuracy is: 0.5366666666666666threshold is: 0.5
Accuracy is: 0.56threshold is: 0.55
Accuracy is: 0.5566666666666666threshold is: 0.6
Accuracy is: 0.55threshold is: 0.65
Accuracy is: 0.5566666666666666threshold is: 0.7
Accuracy is: 0.55threshold is: 0.75
Accuracy is: 0.55threshold is: 0.8
Accuracy is: 0.5333333333333333threshold is: 0.85
Accuracy is: 0.5433333333333333threshold is: 0.9
Accuracy is: 0.5466666666666666threshold is: 0.95


In [84]:
for i in vals.values:
    print(len(i))


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [61]:
input_array = ['Galaxy Wizard' , 'Galaxy Soldier' , 'Photon Orbital' , 'Galaxy-Eyes Photon Dragon' ,'Galaxy Summoner']
    
decider = tf.convert_to_tensor([stitcher(0 , [] , False, input_array)])
subject_card = tf.convert_to_tensor([np.concatenate( (Tokenized_sequence_database[4281] , np.zeros(724)) , axis=None )])

Model((decider , subject_card))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.9022968]], dtype=float32)>

In [ ]:
def predict_matches():
    #input_array = [input('enter') for _ in range(5)]
    input_array = ['Galaxy Wizard' , 'Galaxy Soldier' , 'Photon Orbital' , 'Galaxy-Eyes Photon Dragon' ,'Galaxy Summoner']
    
    decider = tf.convert_to_tensor([stitcher(0 , [] , False, input_array)])
    count = 0
    for indx in pd.DataFrame(Tokenized_sequence_database).index.values:
        if count<0:
            pass
        else:
            if (count % 100) == 0:
                print(count)
            subject_card = tf.convert_to_tensor([np.concatenate( (Tokenized_sequence_database[indx] , np.zeros(724)) , axis=None )])
            
            
            if Model.predict((decider , subject_card) , verbose=False)[0][0] > 0.5:
                print(Sliced_df[indx:indx+1]['name'])
                
                
        count+=1
predict_matches()


In [141]:
pred = pd.DataFrame(Model.predict(Experimentation_Gen()))
count = 0
for i in pred[0]:
    if i > 0.5:
        count+=1
print(count/len(pred))


504/504 [==============================] - 8s 12ms/step
0.16468253968253968


In [220]:
k , trash = next(Experimentation_Gen())
l1 , l2 = k
singular_pred = Model.predict(k)
singular_pred

1/1 [==============================] - 0s 219ms/step


array([[0.9966325]], dtype=float32)

In [222]:

tokenizer.sequences_to_texts([l2[0].numpy()])

["2 aqua effect monster water cannot be used as a synchro material. this card's name becomes des frog while it is on the field. if this card is in your graveyard: you can banish 1 frog monster from your graveyard; special summon this card."]

In [17]:
Tokenized_sequence_database[4281]

array([1065,   -1,  398,    0,   -1,  294,   20,    8,    0,   -1,  251,
         -1, 1618,   10,    5,  731,    0,    0,    0,    0,    0,   -1,
        200,    1,    6,   87,    7,   10,  231,  508,   78,  134,   83,
        315,   78,  129,   89,   75,   22,  113,   76,   38,    5,   44,
          9,    5,   47,   78,   58,   89,   32,    3,   42,  262,   78,
        384,  125,    3,   48,   29,   98,  197,    7,    4,   13,  140,
         29,  188,    5,   92,    1,    6,   24,   31,    2,   20,   10,
       1618,   10,    5,  731,   18,   19,   25,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [7]:
Model = FullModel(100 , 12647 , 1000 , 0.1 , 2)
writer = tf.summary.create_file_writer('Logs/GraphViz')

input_array = ['Galaxy Wizard' , 'Galaxy Soldier' , 'Photon Orbital' , 'Galaxy-Eyes Photon Dragon' ,'Galaxy Summoner']

decider1 = tf.convert_to_tensor([stitcher(0 , [] , False, input_array)])
subject_card1 = tf.convert_to_tensor(    [np.concatenate( (Tokenized_sequence_database[7] , np.zeros(724)) , axis=None )]     ) 

@tf.function
def my_func(x):
    return Model(x)

tf.summary.trace_on(graph=True,profiler=True)
out = my_func((decider1 , subject_card1))

with writer.as_default():
    tf.summary.trace_export(
        name='function_Trace',
        step=0,
        profiler_outdir='Logs\\GraphViz\\'

    )



Instructions for updating:
use `tf.profiler.experimental.start` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
